In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'
import gzip
from collections import defaultdict
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LinearRegression
import string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import os
import pandas as pd
from itertools import chain
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity, laplacian_kernel
from sklearn.decomposition import PCA
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import accuracy_score

In [2]:
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

# allHelpful = []
# userHelpful = defaultdict(list)

# for l in readGz("train.json.gz"):
#   user,item = l['reviewerID'],l['itemID']
#   allHelpful.append(l['helpful'])
#   userHelpful[user].append(l['helpful'])

# averageRate = sum([x['nHelpful'] for x in allHelpful]) * 1.0 / sum([x['outOf'] for x in allHelpful])
# userRate = {}
# for u in userHelpful:
#   totalU = sum([x['outOf'] for x in userHelpful[u]])
#   if totalU > 0:
#     userRate[u] = sum([x['nHelpful'] for x in userHelpful[u]]) * 1.0 / totalU
#   else:
#     userRate[u] = averageRate

In [3]:
import pandas as pd
from collections import defaultdict

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

# df = getDF('train.json.gz')
# test_df = getDF('test_Helpful.json.gz')

## Implement a simple model and calculate MAE

In [43]:
%%time
df = getDF('train.json.gz')

CPU times: user 24.6 s, sys: 2.3 s, total: 26.9 s
Wall time: 28.1 s


In [44]:
df.reviewText[0]

'These are cute, but they are a little small.  When I put them on, my legs stretch the fabric making the black fade out.'

In [45]:
test_df = getDF('test_Helpful.json.gz')

In [46]:
test_df.head()

,categoryID,categories,itemID,reviewerID,rating,reviewText,reviewHash,reviewTime,summary,unixReviewTime,helpful,price
0,0,"[[Sports & Outdoors, Other Sports, Dance, Clot...",I520932398,U816789534,3.0,I ordered according to the size chart but it's...,R157684793,"07 15, 2011",Too small,1310688000,{'outOf': 2},NaN
1,0,"[[Sports & Outdoors, Clothing, Women, Hoodies]...",I969532331,U987148846,4.0,Super thin but really cute and not cheap-looki...,R732719858,"07 17, 2013",Fun hoodie,1374019200,{'outOf': 0},NaN
2,0,"[[Clothing, Shoes & Jewelry, Women, Accessorie...",I149943341,U628436634,5.0,"It was a present for my sis, and she loves Fle...",R352659313,"12 8, 2013",A Perfect hook up,1386460800,{'outOf': 1},NaN
3,0,"[[Clothing, Shoes & Jewelry, Women, Accessorie...",I909025835,U924107228,5.0,I love this thing! I guess they don't make th...,R277416618,"11 22, 2012",I love this thing...,1353542400,{'outOf': 1},7.51
4,0,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",I228439768,U060135484,4.0,I liked it and I wear it...it's a little bit s...,R645892076,"04 1, 2014",I liked it...,1396310400,{'outOf': 0},NaN


In [47]:
# df_features = df.drop(['helpful'], axis=1)
df_features = df
df_labels = df['helpful']

In [48]:
X_train_raw, X_val_raw, y_train_raw, y_val_raw = train_test_split(df_features, df_labels, 
                                                                  test_size=0.25, random_state=10)

### Baseline Model - Finds user rate for helpfulness

In [10]:
# calculate MAE for baseline on validation data

# find user rates for training data
allHelpful = []
userHelpful = defaultdict(list)

user,item = X_train_raw['reviewerID'], X_train_raw['itemID']
allHelpful = y_train_raw.values

for x,y in zip(user, allHelpful):
     userHelpful[x].append(y)

averageRate = sum([x['nHelpful'] for x in allHelpful]) * 1.0 / sum([x['outOf'] for x in allHelpful])

userRate = {}
for u in userHelpful:
    totalU = sum([x['outOf'] for x in userHelpful[u]])
    if totalU > 0:
        userRate[u] = sum([x['nHelpful'] for x in userHelpful[u]]) * 1.0 / totalU
    else:
        userRate[u] = averageRate
        
# predict nHelpful on validation data
pred = []
for u,i,outOf in zip(X_val_raw['reviewerID'], X_val_raw['itemID'], y_val_raw):
    if u in userRate:
        pred.append((outOf['outOf']*userRate[u]))
    else:
        pred.append((outOf['outOf']*averageRate))
        
# calculate MAE 
print('Baseline Model')
print('MAE for val data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_val_raw], pred)))

Baseline Model
MAE for val data: 0.2633751340380996


### Test Multiple models on validation data

In [11]:
def GetPosNegWords_count_Rate(text):
        
    # import positive and negative word lists, define as set for higher efficiency
    posWords_list = set([i.strip() for i in open("positive-words.txt", "r").readlines()])
    negWords_list = set([i.strip() for i in open("negative-words.txt", "r", encoding="ISO-8859-1").readlines()])
    
    # count the number of positive and negative words present in each review text
    dict_ = defaultdict(list)
    for review_text in text:
        
        # remove punctuation symbols and spaces
        words = [n.lower().translate(str.maketrans('','',string.punctuation)) for n in review_text.split(' ')]
        words = [i for i in words if i != ''] # remove spaces
        
        # count number of positive and negative words in each review
        pos_count, neg_count = 0, 0
        for word in words:
            if word in posWords_list:
                pos_count+=1
            elif word in negWords_list:
                neg_count+=1
            else: 
                continue
        
        # save count and rate 
        if len(words) == 0: # prevent division by zero
            dict_['pos_count'].append(-1)
            dict_['neg_count'].append(-1)
            dict_['pos_rate'].append(-1)
            dict_['neg_rate'].append(-1)
        else:
            dict_['pos_count'].append(pos_count)
            dict_['neg_count'].append(neg_count)
            dict_['pos_rate'].append(pos_count/len(words))
            dict_['neg_rate'].append(neg_count/len(words))
    
    return dict_

In [12]:
def Get_NumWords(text):
    
    # get stopwords from nltk library
#     nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    
    # loop over each review
    list_ = []
    for review_text in text:
        
        # remove punctuation symbols and spaces
        words = [n.lower().translate(str.maketrans('','',string.punctuation)) for n in review_text.split(' ')]
        words = [i for i in words if i != ''] # remove spaces
                
        # count number of words, excluding stopwords
        word_count = 0
        for word in words:
            if word in stop_words:
                continue
            else:
                 word_count+=1
                    
        # save counts
        list_.append(word_count)
        
    return list_      

In [13]:
def Get_category_numtrans(category_lists):
    
    # get unique category lists
    lists = []
    for i in category_lists:
        for j in i:
            lists.append(tuple(j))

    # make dictionary with IDs
    categorylists_dict = {k:v for v,k in enumerate(set(lists))}

    # transform list occurance to numbers using dictionary 
    category_numtrans = []
    for i in category_lists:
        sum_ = 0
        for j in i:
            sum_ += categorylists_dict[tuple(j)]
        category_numtrans.append(sum_)

    # return transformation
    return category_numtrans

In [14]:
def get_helpfulRate(col):
    
    allHelpful = []
    colHelpful = defaultdict(list)

    col_data = X_train_raw[col]
    allHelpful = y_train_raw

    for x,y in zip(col_data, allHelpful):
         colHelpful[x].append(y)

    averageRate = sum([x['nHelpful'] for x in allHelpful]) * 1.0 / sum([x['outOf'] for x in allHelpful])

    rate = {}
    for u in colHelpful:
        totalU = sum([x['outOf'] for x in colHelpful[u]])
        if totalU > 0:
            rate[u] = sum([x['nHelpful'] for x in colHelpful[u]]) * 1.0 / totalU
        else:
            rate[u] = averageRate 
    
    return rate, averageRate

In [15]:
def find_Helpful_rate(data, colname):
    
    rate_dict, avg_rate = get_helpfulRate(colname)

    ratehelpful = []
    for i in data:
        # use average for entries not present
        try:
            ratehelpful.append(rate_dict[i])
        except:
            ratehelpful.append(avg_rate)
            
    return ratehelpful

In [16]:
def Get_punctuation_count(data):

    punct_set = set(string.punctuation)
    list_ = []
    for str_ in data:
        count_ = 0
        for c in str_:
            if c in punct_set:
                count_+=1
            else:
                continue
        list_.append(count_)
        
    return list_

In [17]:
def Get_numreviews_summarized(data):

    # initialize dict
    dict_ = {k:0 for k in set(data)}
    
    # count number 
    for r in data:
        dict_[r]+=1
        
    # create list
    list_ = []
    for r in data:
        try:
            list_.append(dict_[r])
        except:
            list_.append(0)
    
    return list_

In [18]:
def Get_product_ratingDeviation(df):

    ProdRating_mean_dict = df[['itemID', 'rating']].groupby(['itemID']).mean().rating.to_dict()

    rating_deviation = []
    for r, p in zip(df.rating, df.itemID):
        try:
            rating_deviation.append(r - ProdRating_mean_dict[p])
        except:
            rating_deviation.append(-444)
        
    return rating_deviation

In [19]:
def Get_count_firstLetterCapital(data):
    
    list_ = []
    for review in data:
        count_ = 0
        for word in review.split():
            if word[0].isupper():
                count_+=1
        list_.append(count_)
        
    return list_


In [20]:
def Get_avg_word_length(data):
    
    # get stopwords from nltk library
#     nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    
    # loop over each review
    list_ = []
    for review_text in data:
        
        # remove punctuation symbols and spaces
        words = [n.lower().translate(str.maketrans('','',string.punctuation)) for n in review_text.split(' ')]
        words = [i for i in words if i != ''] # remove spaces
                
        # remove stopwords
        words = set(words) - stop_words
                    
        # save average length
        try:
            list_.append(sum([len(w) for w in words])/len(words))
        except:
            list_.append(-1)
        
    return list_

In [21]:
def Get_delta_sinceFirstReview(df):
    
    # find time for first review in each product
    first_product_reviewtime = df[['itemID','unixReviewTime']].groupby(['itemID']).min().unixReviewTime.to_dict()

    # find delta for each review
    delta = []
    for i,t in zip(df.itemID, df.unixReviewTime):
        delta.append(t-first_product_reviewtime[i])
        
    return delta

In [22]:
def Get_delta_sinceLastReview(df):
    
    # create reduced df
    new_df = df[['itemID','unixReviewTime']].groupby(['itemID'])['unixReviewTime'].apply(list)

    # create dictionary with deltas
    delta_dict = defaultdict(dict)
    
    for idx, times_list in zip(new_df.index, new_df):

        times_list = sorted(times_list)
        times_list_deltas = np.append(np.array([0]) , np.diff(times_list))

        for t, d in zip(times_list, times_list_deltas):
            delta_dict[idx][t] = d

    # generate list with deltas matching input dataframe
    list_ = []
    for u,t in zip(df.itemID, df.unixReviewTime):
        list_.append(delta_dict[u][t])
        
    return list_

In [23]:
def Get_colsRatio(col1, col2):
    
    list_ = []
    for i,j in zip(col1, col2):

        try:
            list_.append(i/j)
        except:
            list_.append(-1)

    return list_

In [24]:
def Get_numCapitalwords(data):
    
    # get stopwords from nltk library
#     nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    
    # loop over each review
    list_ = []
    for review_text in data:
        
        # remove punctuation symbols and spaces
        words = [n.translate(str.maketrans('','',string.punctuation)) for n in review_text.split(' ')]
        words = [i for i in words if i != ''] # remove spaces
                
        # remove stopwords
        words = set(words) - stop_words
        
        # append count of upper case words
        list_.append(len([i for i in words if i.isupper()]))
        
    return list_

In [25]:
def Get_ExclQues_charCount(data):
    
    # loop over each review
    list_ = []
    for review_text in data:
        # append count of question and exclamation characters
        list_.append(review_text.count('?') + review_text.count('!'))
        
    return list_

In [26]:
# def Get_TFIDF(text):
    
#     # get stopwords from nltk library
#     nltk.download('stopwords')
#     stopW_list = stopwords.words('english')
    
#     # create the transform
#     vectorizer = TfidfVectorizer(stop_words = stopW_list, lowercase=True, smooth_idf=True)
    
#     # tokenize and build vocab
#     vectorizer.fit(text)
    
#     # encode document
#     vector = vectorizer.transform(text)
    
#     return pd.DataFrame(vector.toarray())

In [27]:
def Get_features(df):
    
    # Modify ----------> "categories"
    
    # get number of characters
    df.loc[:,'categories_count'] = [len(i) for i in df['categories']]
    
#     # generate numerical category by transforming combination of lists to numbers
    df.loc[:,'category_numtrans'] = Get_category_numtrans(df.categories)

    # Modify ----------> "itemID" and "reviewerID"
    
    # create dictionaries for itemID and reviewerID, convert from categorical to numeric
    items_dict = {k:v for v,k in enumerate(set(df.itemID))}
    reviewer_dict = {k:v for v,k in enumerate(set(df.reviewerID))}
    
    # change item and reviewer IDs to numeric
    df.loc[:,'itemID'] = [items_dict[i] for i in df['itemID']]
    df.loc[:,'reviewerID'] = [reviewer_dict[i] for i in df['reviewerID']]
    
    # add helpful rate for itemID and reviewerID
    df.loc[:,'itemID_helpfulRate'] = find_Helpful_rate(df['itemID'], 'itemID')
    df.loc[:,'reviewerID_helpfulRate'] = find_Helpful_rate(df['reviewerID'], 'reviewerID')
    
    # get number of reviews for each user
    df.loc[:,'reviewerID_numReviews'] = Get_numreviews_summarized(df['reviewerID'])
    
    # get number of reviews for each product
    df.loc[:,'itemID_numReviews'] = Get_numreviews_summarized(df['itemID'])
    
    # Modify ----------> "reviewText" 
    
    # get number of words, remove stopwords
    df.loc[:,'reviewText_count_words'] = Get_NumWords(df['reviewText'])

    # get character count
    df.loc[:,'reviewText_count_char'] = [len(i) for i in df['reviewText']]
    
    # get punctuation count
    df.loc[:,'reviewText_count_punctu'] = Get_punctuation_count(df['reviewText'])
    
    # get number of words that start with a capital leter
    df.loc[:,'reviewText_count_firstCapital'] = Get_count_firstLetterCapital(df['reviewText'])
    
    # get average word length
    df.loc[:,'reviewText_avgWordLength'] = Get_avg_word_length(df['reviewText'])
    
    # get number of capital words
    df.loc[:,'reviewText_capitalwords'] = Get_numCapitalwords(df['reviewText'])
    
    # get number of question and exclamation characters
    df.loc[:,'reviewText_ExclQue_countchar'] = Get_ExclQues_charCount(df['reviewText'])
    
    # get ratio between puctuations with character numbers
    df.loc[:,'reviewText_PunctChar_ratio'] = Get_colsRatio(df['reviewText_count_punctu'], df['reviewText_count_char'])
    
    # get positive and negative word rate
    reviewText_PosNeg = GetPosNegWords_count_Rate(df.reviewText)
    df.loc[:,'reviewText_posWordCount'] = reviewText_PosNeg['pos_count']
    df.loc[:,'reviewText_negWordCount'] = reviewText_PosNeg['neg_count']
    df.loc[:,'reviewText_posWordRate'] = reviewText_PosNeg['pos_rate']
    df.loc[:,'reviewText_negWordRate'] = reviewText_PosNeg['neg_rate']
  
    # Modify ----------> "summary"
    
    # get number of words, remove stopwords
    df.loc[:,'summary_count_words'] = Get_NumWords(df['summary'])

    # get character count
    df.loc[:,'summary_count_char'] = [len(i) for i in df['summary']]
    
    # get punctuation count
    df.loc[:,'summary_count_punctu'] = Get_punctuation_count(df['summary'])
    
    # get number of words that start with a capital leter
    df.loc[:,'summary_count_firstCapital'] = Get_count_firstLetterCapital(df['summary'])
    
    # get average word length
    df.loc[:,'summary_avgWordLength'] = Get_avg_word_length(df['summary'])
    
    # get number of capital words
    df.loc[:,'summary_capitalwords'] = Get_numCapitalwords(df['summary'])
    
    # get number of question and exclamation characters
    df.loc[:,'summary_ExclQue_countchar'] = Get_ExclQues_charCount(df['summary'])
    
    # get ratio between puctuations with character numbers
    df.loc[:,'summary_PunctChar_ratio'] = Get_colsRatio(df['summary_count_punctu'], df['summary_count_char'])
    
    # get positive and negative word rate
    summary_PosNeg = GetPosNegWords_count_Rate(df.summary)
    df.loc[:,'summary_posWordCount'] = summary_PosNeg['pos_count']
    df.loc[:,'summary_negWordCount'] = summary_PosNeg['neg_count']
    df.loc[:,'summary_posWordRate'] = summary_PosNeg['pos_rate']
    df.loc[:,'summary_negWordRate'] = summary_PosNeg['neg_rate']
    
    # Modify ----------> "helpful" 
    
    # parse helpful votes
    df.loc[:,'outOf_feature'] = [i['outOf'] for i in df['helpful']]
    
    # Modify ----------> "price" 
    
    # change NA values to -999
    df.loc[df.price.isna(),'price'] = -999
    
    # Modify ----------> "unixReviewTime" 
    
    # find time since first review
    df.loc[:,'unixReviewTime_delta_firstreview'] = Get_delta_sinceFirstReview(df)
    
    # find time since last review for same product
    df.loc[:,'unixReviewTime_delta_lastreview'] = Get_delta_sinceLastReview(df)
    
    # normalize UnixReviewTime
#     df.loc[:,'unixReviewTime'] = ((df.unixReviewTime-df.unixReviewTime.mean())/df.unixReviewTime.std())
    
    # Modify ----------> "reviewText" 
    
    # get rating deviation from the mean
    df.loc[:,'rating_deviation'] = Get_product_ratingDeviation(df)
    
    # Add -------------> New Columns
    
    # votes over time
    df.loc[:,'votes_time'] = df.outOf_feature/df.unixReviewTime
    
    # ratio of summary to reviewText for characters and words
    df.loc[:,'summary_reviewText_charRatio'] = Get_colsRatio(df.summary_count_char, df.reviewText_count_char)
    df.loc[:,'summary_reviewText_wordsRatio'] = Get_colsRatio(df.summary_count_words, df.reviewText_count_words)
    
    # define columns to keep
    cols = ['categoryID','categories_count', 'category_numtrans','summary_count_words', 'itemID', 'reviewerID', 'rating', 
            'reviewText_count_words', 'itemID_helpfulRate', 'reviewerID_helpfulRate',
            'reviewerID_numReviews', 'rating_deviation', 
            'outOf_feature', 'unixReviewTime', 'price', 'reviewText_posWordCount', 'reviewText_negWordCount',
            'reviewText_posWordRate', 'reviewText_negWordRate', 'summary_count_char', 'reviewText_count_char',
            'reviewText_count_punctu','summary_count_punctu','summary_posWordCount', 
            'summary_negWordCount', 'summary_posWordRate','summary_negWordRate',
           'reviewText_count_firstCapital', 'summary_count_firstCapital',
           'reviewText_avgWordLength','summary_avgWordLength', 'unixReviewTime_delta_firstreview',
           'votes_time', 'summary_reviewText_charRatio', 'summary_reviewText_wordsRatio', 'itemID_numReviews',
           'reviewText_capitalwords', 'summary_capitalwords',
           'reviewText_ExclQue_countchar', 'summary_ExclQue_countchar',
           'reviewText_PunctChar_ratio', 'summary_PunctChar_ratio', 'unixReviewTime_delta_lastreview']
    
    # return features
    return df[cols]

In [28]:
def Get_labels(df):    
    return [i['nHelpful']/i['outOf'] if i['outOf']!=0 else 0 for i in df]

### Apply TF-IDF

In [29]:
# %%time
# # add TF-IDF for reviewText

# # get stopwords from nltk library
# # nltk.download('stopwords')
# stopW_list = stopwords.words('english')

# # create the transform
# vectorizer = TfidfVectorizer(stop_words = stopW_list, lowercase=True, smooth_idf=True)

# # tokenize and build vocab
# vectorizer.fit(X_train_raw.summary)

# # encode document
# vector_train = vectorizer.transform(X_train_raw.summary)
# vector_val = vectorizer.transform(X_val_raw.summary)

# # convert to dataframe
# vector_train = pd.DataFrame(vector_train.toarray())
# vector_val = pd.DataFrame(vector_val.toarray())

In [30]:
# %%time

# # Get features
# X_train_features = Get_features(X_train_raw)
# y_train = Get_labels(y_train_raw) # continuous
# # y_train = [int(i*100) for i in Get_labels(y_train_raw)] # categorical

# X_val_features = Get_features(X_val_raw)
# y_val = y_val_raw


# # combine features with TF-IDF
# X_train = vector_train
# for i in X_train_features.columns:
#     X_train[i] = X_train_features[i].tolist()
    
# X_val = vector_val
# for i in X_val_features.columns:
#     X_val[i] = X_val_features[i].tolist()

### Get features for train and val

In [139]:
# %%time
# from itertools import compress

# # get indices from raw dataset where number of votes is greater than zero
# idx_train = np.array([i['outOf'] for i in X_train_raw.helpful]) > 0

# # filter raw data for training only
# X_train_raw = X_train_raw.loc[idx_train,:]
# y_train_raw = list(compress(y_train_raw, idx_train))

# # get training features and labels
# X_train_1 = Get_features(X_train_raw)
# y_train = Get_labels(y_train_raw) 

# # get validation features and labels
# X_val_1 = Get_features(X_val_raw)
# y_val = y_val_raw

CPU times: user 1min 33s, sys: 1.49 s, total: 1min 34s
Wall time: 1min 35s


In [32]:
%%time
from itertools import compress

# get indices from raw dataset where number of votes is greater than zero
idx_train = np.array([i['outOf'] for i in X_train_raw.helpful]) > 0

# get training features and labels
X_train_filtered = Get_features(X_train_raw.loc[idx_train,:])
y_train_filtered = Get_labels(list(compress(y_train_raw, idx_train))) 

X_train = Get_features(X_train_raw)
y_train = Get_labels(y_train_raw)

# get validation features and labels
X_val = Get_features(X_val_raw)
y_val = y_val_raw

CPU times: user 3min 37s, sys: 2.7 s, total: 3min 40s
Wall time: 3min 44s


In [33]:
X_train.head()

,categoryID,categories_count,category_numtrans,summary_count_words,itemID,reviewerID,rating,reviewText_count_words,itemID_helpfulRate,reviewerID_helpfulRate,...,summary_reviewText_charRatio,summary_reviewText_wordsRatio,itemID_numReviews,reviewText_capitalwords,summary_capitalwords,reviewText_ExclQue_countchar,summary_ExclQue_countchar,reviewText_PunctChar_ratio,summary_PunctChar_ratio,unixReviewTime_delta_lastreview
130396,0,3,3192,2,16908,30819,5.0,24,1.000000,0.857143,...,0.044248,0.083333,5,1,0,1,0,0.048673,0.0000,0
86666,1,3,2696,1,8006,13591,5.0,11,1.000000,1.000000,...,0.042735,0.090909,15,0,0,0,0,0.042735,0.0000,2073600
94994,0,1,171,1,9127,208,4.0,14,0.833333,0.818182,...,0.054422,0.071429,4,1,0,0,0,0.020408,0.0000,0
104230,0,1,758,4,6082,6757,5.0,13,0.900000,0.500000,...,0.141892,0.307692,21,1,0,0,0,0.013514,0.0000,24624000
194871,0,2,2252,5,18149,27172,5.0,157,0.923077,0.812500,...,0.020343,0.031847,40,3,0,2,0,0.044501,0.0625,259200


In [56]:
# %%time

# # Get features
# X_train_1 = Get_features(X_train_raw)
# y_train = Get_labels(y_train_raw) # continuous
# # y_train = [int(i*100) for i in Get_labels(y_train_raw)] # categorical

# X_val_1 = Get_features(X_val_raw)
# y_val = y_val_raw

In [57]:
# # remove reviews where votes are zero - training data only
# from itertools import compress

# # idx_train = (X_train_1.outOf_feature > 0) & (X_train_1.outOf_feature < 3)
# idx_train = X_train_1.outOf_feature > 0
# X_train_1 = X_train_1.loc[idx_train,:]
# y_train = list(compress(y_train, idx_train))
# X_train_raw = X_train_raw.loc[idx_train,:]
# y_train_raw = list(compress(y_train_raw, idx_train))

# # idx_val = (X_val_1.outOf_feature > 0) & (X_val_1.outOf_feature < 3)
# idx_val = X_val_1.outOf_feature > 0
# X_val_1 = X_val_1.loc[idx_val,:]
# y_val = list(compress(y_val, idx_val))
# X_val_raw = X_val_raw.loc[idx_val,:]
# y_val_raw = list(compress(y_val_raw, idx_val))

In [84]:
X_train_1.head()

,categoryID,categories_count,category_numtrans,summary_count_words,itemID,reviewerID,rating,reviewText_count_words,itemID_helpfulRate,reviewerID_helpfulRate,...,summary_reviewText_charRatio,summary_reviewText_wordsRatio,itemID_numReviews,reviewText_capitalwords,summary_capitalwords,reviewText_ExclQue_countchar,summary_ExclQue_countchar,reviewText_PunctChar_ratio,summary_PunctChar_ratio,unixReviewTime_delta_lastreview
130248,0,2,551,3,15265,20193,4.0,108,0.793464,0.714286,...,0.022203,0.027778,49,1,0,2,0,0.021349,0.038462,4924800
182454,0,1,437,3,10110,20969,2.0,9,0.750000,0.625000,...,0.234783,0.333333,5,1,0,0,0,0.043478,0.111111,1036800
176029,0,1,327,1,2813,6219,5.0,14,0.695652,0.000000,...,0.076433,0.071429,10,0,0,1,1,0.025478,0.083333,172800
13049,0,1,1104,1,11768,14780,5.0,30,0.897436,0.882979,...,0.024922,0.033333,5,1,0,1,0,0.037383,0.000000,86400
25017,0,2,2276,2,5445,1962,1.0,19,0.428571,0.500000,...,0.057471,0.105263,4,2,0,0,0,0.030651,0.066667,5097600


In [85]:
X_val_1.head()

,categoryID,categories_count,category_numtrans,summary_count_words,itemID,reviewerID,rating,reviewText_count_words,itemID_helpfulRate,reviewerID_helpfulRate,...,summary_reviewText_charRatio,summary_reviewText_wordsRatio,itemID_numReviews,reviewText_capitalwords,summary_capitalwords,reviewText_ExclQue_countchar,summary_ExclQue_countchar,reviewText_PunctChar_ratio,summary_PunctChar_ratio,unixReviewTime_delta_lastreview
188807,0,1,879,1,14635,8560,5.0,13,0.812500,1.000000,...,0.076271,0.076923,13,1,0,0,0,0.025424,0.000000,9590400
199122,0,2,2359,1,14729,10725,3.0,27,0.944444,1.000000,...,0.035503,0.037037,2,1,0,0,0,0.011834,0.083333,0
17944,1,2,1143,1,11532,619,5.0,5,0.739130,0.000000,...,0.117647,0.200000,8,0,0,0,0,0.000000,0.000000,11923200
199504,1,4,2755,4,7407,11173,5.0,45,0.625000,0.922727,...,0.060194,0.088889,6,1,0,9,0,0.034951,0.000000,10022400
168703,0,1,1086,2,9815,17197,5.0,39,1.000000,0.571429,...,0.028902,0.051282,1,1,0,1,0,0.034682,0.000000,0


In [279]:
X_train_1.shape

(32485, 31)

In [280]:
X_val_1.shape

(8088, 31)

### Exclude certain features

In [118]:
"""
Testing was done with random forest regressor
RandomForestRegressor(max_depth=C, random_state=0)
"""

# data from help votes
cols1 = ['itemID', 'reviewerID','outOf_feature']

"""
10
MAE for train data: 0.1763875
MAE for val data: 0.184775
CPU times: user 15.2 s, sys: 250 ms, total: 15.5 s
Wall time: 15.6 s
"""

# raw features from data
cols2 = ['itemID', 'reviewerID', 'rating', 'outOf_feature', 'unixReviewTime', 'price']

"""
10
MAE for train data: 0.16409375
MAE for val data: 0.1682
CPU times: user 20.1 s, sys: 212 ms, total: 20.3 s
Wall time: 20.4 s

with unixReviewTime normalized:
10
MAE for train data: 0.1641
MAE for val data: 0.169225
CPU times: user 20.7 s, sys: 433 ms, total: 21.1 s
Wall time: 21.3 s
"""

# # new features around word manipulations in reviewText and summary
# cols3 = ['itemID', 'reviewerID', 'reviewText_count_words',
#        'reviewText_posWordCount', 'reviewText_negWordCount',
#        'reviewText_posWordRate', 'reviewText_negWordRate', 'summary_count_words']

# """
# 10
# MAE for train data: 0.68605625
# MAE for val data: 0.714175
# CPU times: user 1min, sys: 803 ms, total: 1min 1s
# Wall time: 1min 2s

# # counting number of characters
# 10
# MAE for train data: 0.68730625
# MAE for val data: 0.718175
# CPU times: user 1min 1s, sys: 424 ms, total: 1min 1s
# Wall time: 1min 1s
# """

# # new features around word manipulations in reviewText and summary, without pos/neg rates
# cols4 = ['itemID', 'reviewerID', 'reviewText_count_words',
#        'reviewText_posWordCount', 'reviewText_negWordCount', 'summary_count_words']

# """
# 10
# MAE for train data: 0.6877375
# MAE for val data: 0.715225
# CPU times: user 46 s, sys: 309 ms, total: 46.3 s
# Wall time: 46.5 s
# """

# # all features
# cols5 = ['categories_count', 'summary_count_words', 'itemID', 'reviewerID', 'rating',
#        'reviewText_count_words', 'outOf_feature', 'unixReviewTime', 'price',
#        'reviewText_posWordCount', 'reviewText_negWordCount',
#        'reviewText_posWordRate', 'reviewText_negWordRate']

# """
# 10
# MAE for train data: 0.1636125
# MAE for val data: 0.1661
# CPU times: user 34 s, sys: 313 ms, total: 34.3 s
# Wall time: 34.5 s
# """


cols6 = ['itemID', 'reviewerID', 'rating', 'outOf_feature', 'unixReviewTime', 'price', 'reviewText_count_words']



# try features elimination
all_ = ['categoryID','categories_count', 'category_numtrans','summary_count_words', 'itemID', 'reviewerID', 'rating', 
            'reviewText_count_words', 'itemID_helpfulRate', 'reviewerID_helpfulRate',
            'reviewerID_numReviews', 'rating_deviation', 
            'outOf_feature', 'unixReviewTime', 'price', 'reviewText_posWordCount', 'reviewText_negWordCount',
            'reviewText_posWordRate', 'reviewText_negWordRate', 'summary_count_char', 'reviewText_count_char',
            'reviewText_count_punctu','summary_count_punctu','summary_posWordCount', 
            'summary_negWordCount', 'summary_posWordRate','summary_negWordRate',
           'reviewText_count_firstCapital', 'summary_count_firstCapital',
           'reviewText_avgWordLength','summary_avgWordLength']


X_train = X_train_1[all_]
X_val = X_val_1[all_]


In [127]:
X_train.head()

,categoryID,categories_count,category_numtrans,summary_count_words,itemID,reviewerID,rating,reviewText_count_words,itemID_helpfulRate,reviewerID_helpfulRate,...,reviewText_count_punctu,summary_count_punctu,summary_posWordCount,summary_negWordCount,summary_posWordRate,summary_negWordRate,reviewText_count_firstCapital,summary_count_firstCapital,reviewText_avgWordLength,summary_avgWordLength
62732,0,3,3392,2,16904,18201,5.0,17,1.000000,0.850732,...,4,0,0,1,0.0,0.333333,5,0,5.941176,4.0
182454,0,1,161,3,2023,28682,2.0,9,0.750000,0.625000,...,5,3,0,0,0.0,0.000000,4,1,5.000000,5.0
158745,0,1,161,1,1940,11231,4.0,25,0.500000,0.800000,...,3,0,1,0,1.0,0.000000,9,1,5.318182,4.0
176029,0,1,756,1,12868,14824,5.0,14,0.695652,0.000000,...,4,1,1,0,0.5,0.000000,3,0,6.000000,6.0
64520,0,3,1715,2,6313,12493,5.0,22,0.500000,0.000000,...,5,1,1,0,0.5,0.000000,8,1,5.045455,6.0


In [129]:
X_train.shape

(132424, 31)

### Test Feature Contribution to MAE LR

In [134]:
%%time

#################################### Baseline ####################################
# define original feature lists
original_features = ['itemID', 'reviewerID', 'outOf_feature']

# apply logistic regression to votes equal to 0 and 1

X_train = X_train_1[original_features]
X_val = X_val_1[original_features]

# parameters = { 'penalty': ['l1','l2'], 
#               'C':[0.1, 0.5, 1, 2, 3, 4, 5, 10]}
clf = LogisticRegression(penalty='l2', C=1.0, n_jobs=-1)
# clf = GridSearchCV(logreg, parameters, verbose=True, n_jobs=-1)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_val)*np.array([i['outOf'] for i in X_val_raw['helpful']])
y_pred_train = clf.predict(X_train)*np.array([i['outOf'] for i in X_train_raw['helpful']])

# round to the nearest integer
y_pred = [int(round(i)) for i in y_pred]
y_pred_train = [int(round(i)) for i in y_pred_train]

print('Baseline Features')
print('Accuracy for train data: {}'.format(accuracy_score([i['nHelpful'] for i in y_train_raw],y_pred_train)))
print('Accuracy for val data: {}'.format(accuracy_score([i['nHelpful'] for i in y_val_raw],y_pred)))

print('MAE for train data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_train_raw], y_pred_train)))
baseline_mae = mean_absolute_error([i['nHelpful'] for i in y_val_raw], y_pred)
print('MAE for val data: {}'.format(baseline_mae))
print('\n')

#################################### New Features ####################################
# define new feature lists
new_feature_list = ['rating', 'unixReviewTime', 'price',
                    'categoryID','categories_count', 'category_numtrans','summary_count_words', 
                    'reviewText_count_words', 'itemID_helpfulRate', 'reviewerID_helpfulRate',
                    'reviewerID_numReviews', 'rating_deviation', 'reviewText_posWordCount', 
                    'reviewText_negWordCount','reviewText_posWordRate', 'reviewText_negWordRate', 
                    'summary_count_char', 'reviewText_count_char', 'reviewText_count_punctu',
                    'summary_count_punctu','summary_posWordCount', 
                    'summary_negWordCount', 'summary_posWordRate','summary_negWordRate', 
                    'reviewText_count_firstCapital', 'summary_count_firstCapital',
                    'reviewText_avgWordLength','summary_avgWordLength']

features_tokeep = []
for feature in new_feature_list:
    X_train = X_train_1[original_features + [feature]]
    X_val = X_val_1[original_features + [feature]]
    
#     parameters = { 'penalty': ['l1','l2'], 
#               'C':[0.1, 0.5, 1, 2, 3, 4, 5, 10]}
    
    clf = LogisticRegression(penalty='l2', C=1.0, n_jobs=-1)
#     clf = GridSearchCV(logreg, parameters, verbose=True, n_jobs=-1)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_val)*np.array([i['outOf'] for i in X_val_raw['helpful']])
    y_pred_train = clf.predict(X_train)*np.array([i['outOf'] for i in X_train_raw['helpful']])

    # round to the nearest integer
    y_pred = [int(round(i)) for i in y_pred]
    y_pred_train = [int(round(i)) for i in y_pred_train]

    print(feature)
    
    print('Accuracy for train data: {}'.format(accuracy_score([i['nHelpful'] for i in y_train_raw],y_pred_train)))
    print('Accuracy for val data: {}'.format(accuracy_score([i['nHelpful'] for i in y_val_raw],y_pred)))
    
    print('MAE for train data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_train_raw], y_pred_train)))
    feature_mae = mean_absolute_error([i['nHelpful'] for i in y_val_raw], y_pred)
    print('MAE for val data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_val_raw], y_pred)))
    print('Difference from baseline: {}'.format(baseline_mae-feature_mae))
    print('\n')
    
    # keep features with lower MAE than baseline
    if feature_mae < baseline_mae:
        features_tokeep.append(feature)
    

Baseline Features
Accuracy for train data: 0.869668640125657
Accuracy for val data: 0.867717676904956
MAE for train data: 0.13033135987434302
MAE for val data: 0.13228232309504395


rating
Accuracy for train data: 0.9519422461185284
Accuracy for val data: 0.9598924828606807
MAE for train data: 0.04805775388147163
MAE for val data: 0.04010751713931926
Difference from baseline: 0.09217480595572469


unixReviewTime
Accuracy for train data: 0.869668640125657
Accuracy for val data: 0.867717676904956
MAE for train data: 0.13033135987434302
MAE for val data: 0.13228232309504395
Difference from baseline: 0.0


price
Accuracy for train data: 0.8696761916268955
Accuracy for val data: 0.867717676904956
MAE for train data: 0.13032380837310456
MAE for val data: 0.13228232309504395
Difference from baseline: 0.0


categoryID
Accuracy for train data: 0.869668640125657
Accuracy for val data: 0.867717676904956
MAE for train data: 0.13033135987434302
MAE for val data: 0.13228232309504395
Difference from 

In [135]:
# print features that help improve
features_tokeep

['rating',
 'categories_count',
 'reviewText_count_words',
 'reviewerID_numReviews',
 'reviewText_posWordCount',
 'reviewText_count_char',
 'reviewText_count_firstCapital',
 'summary_avgWordLength']

In [136]:
# print features that did not help improve
print(set(new_feature_list)-set(features_tokeep))

{'summary_count_char', 'summary_posWordCount', 'reviewText_avgWordLength', 'reviewText_count_punctu', 'reviewerID_helpfulRate', 'reviewText_negWordRate', 'reviewText_posWordRate', 'unixReviewTime', 'price', 'summary_posWordRate', 'itemID_helpfulRate', 'summary_count_words', 'summary_count_punctu', 'categoryID', 'category_numtrans', 'summary_negWordRate', 'rating_deviation', 'summary_count_firstCapital', 'summary_negWordCount', 'reviewText_negWordCount'}


### Test Feature Contribution to MAE RF

In [37]:
%%time

feature_MAE = []

#################################### Baseline ####################################
# define original feature lists
original_features = ['itemID', 'reviewerID', 'outOf_feature']

X_train_temp = X_train_filtered[original_features]
# X_val_temp = X_val_1[original_features]

regr = RandomForestRegressor(max_depth=10, n_estimators=100, random_state=0, n_jobs=-1)
regr.fit(X_train_temp, y_train_filtered)

y_pred = regr.predict(X_val[original_features])*np.array([i['outOf'] for i in X_val_raw['helpful']])
y_pred_train = regr.predict(X_train[original_features])*np.array([i['outOf'] for i in X_train_raw['helpful']])

# round to the nearest integer
y_pred = [int(round(i)) for i in y_pred]
y_pred_train = [int(round(i)) for i in y_pred_train]

print('Baseline Features')
print('MAE for train data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_train_raw], y_pred_train)))
baseline_mae = mean_absolute_error([i['nHelpful'] for i in y_val_raw], y_pred)
feature_MAE.append(('baseline',baseline_mae))
print('MAE for val data: {}'.format(baseline_mae))
print('\n')

#################################### New Features ####################################
# define new feature lists
new_feature_list = ['rating', 'unixReviewTime', 'price',
                    'categoryID','categories_count', 'category_numtrans','summary_count_words', 
                    'reviewText_count_words', 'itemID_helpfulRate', 'reviewerID_helpfulRate',
                    'reviewerID_numReviews', 'rating_deviation', 'reviewText_posWordCount', 
                    'reviewText_negWordCount','reviewText_posWordRate', 'reviewText_negWordRate', 
                    'summary_count_char', 'reviewText_count_char', 'reviewText_count_punctu',
                    'summary_count_punctu','summary_posWordCount', 
                    'summary_negWordCount', 'summary_posWordRate','summary_negWordRate', 
                    'reviewText_count_firstCapital', 'summary_count_firstCapital',
                    'reviewText_avgWordLength','summary_avgWordLength', 'unixReviewTime_delta_firstreview',
                    'summary_reviewText_charRatio', 'summary_reviewText_wordsRatio',
                    'votes_time', 'itemID_numReviews', 'reviewText_capitalwords', 'summary_capitalwords',
                    'reviewText_ExclQue_countchar', 'summary_ExclQue_countchar',
                    'reviewText_PunctChar_ratio', 'summary_PunctChar_ratio', 'unixReviewTime_delta_lastreview']

features_tokeep = []
for feature in new_feature_list:
    
    X_train_temp = X_train_filtered[original_features + [feature]]
#     X_val = X_val_1[original_features + [feature]]
    
    regr = RandomForestRegressor(max_depth=10, n_estimators=100, random_state=0, n_jobs=-1)
    regr.fit(X_train_temp, y_train_filtered)

    y_pred = regr.predict(X_val[original_features + [feature]])*np.array([i['outOf'] for i in X_val_raw['helpful']])
    y_pred_train = regr.predict(X_train[original_features + [feature]])*np.array([i['outOf'] for i in X_train_raw['helpful']])

    # round to the nearest integer
    y_pred = [int(round(i)) for i in y_pred]
    y_pred_train = [int(round(i)) for i in y_pred_train]

    print(feature)
    
    print('Accuracy for train data: {}'.format(accuracy_score([i['nHelpful'] for i in y_train_raw],y_pred_train)))
    print('Accuracy for val data: {}'.format(accuracy_score([i['nHelpful'] for i in y_val_raw],y_pred)))
    
    print('MAE for train data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_train_raw], y_pred_train)))
    feature_mae = mean_absolute_error([i['nHelpful'] for i in y_val_raw], y_pred)
    feature_MAE.append((feature,feature_mae))
    print('MAE for val data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_val_raw], y_pred)))
    print('Difference from baseline: {}'.format(baseline_mae-feature_mae))
    print('\n')
    
    # keep features with lower MAE than baseline
    if feature_mae < baseline_mae:
        features_tokeep.append(feature)

Baseline Features
MAE for train data: 0.21789333333333333
MAE for val data: 0.18766


rating
Accuracy for train data: 0.8498733333333334
Accuracy for val data: 0.86334
MAE for train data: 0.21588
MAE for val data: 0.17534
Difference from baseline: 0.012319999999999998


unixReviewTime
Accuracy for train data: 0.8476866666666667
Accuracy for val data: 0.85496
MAE for train data: 0.20734
MAE for val data: 0.185
Difference from baseline: 0.0026599999999999957


price
Accuracy for train data: 0.8473666666666667
Accuracy for val data: 0.8538
MAE for train data: 0.21122666666666667
MAE for val data: 0.18532
Difference from baseline: 0.002339999999999981


categoryID
Accuracy for train data: 0.84416
Accuracy for val data: 0.85534
MAE for train data: 0.20857333333333333
MAE for val data: 0.18502
Difference from baseline: 0.0026400000000000035


categories_count
Accuracy for train data: 0.84088
Accuracy for val data: 0.85258
MAE for train data: 0.21888666666666667
MAE for val data: 0.18704
Diff

In [38]:
# print features that help improve
features_tokeep

['rating',
 'unixReviewTime',
 'price',
 'categoryID',
 'categories_count',
 'category_numtrans',
 'summary_count_words',
 'reviewText_count_words',
 'itemID_helpfulRate',
 'reviewerID_helpfulRate',
 'rating_deviation',
 'reviewText_posWordCount',
 'reviewText_negWordCount',
 'reviewText_posWordRate',
 'reviewText_negWordRate',
 'summary_count_char',
 'reviewText_count_char',
 'reviewText_count_punctu',
 'summary_count_punctu',
 'summary_posWordCount',
 'summary_negWordCount',
 'summary_posWordRate',
 'summary_negWordRate',
 'reviewText_count_firstCapital',
 'summary_count_firstCapital',
 'unixReviewTime_delta_firstreview',
 'summary_reviewText_charRatio',
 'votes_time',
 'reviewText_capitalwords',
 'summary_capitalwords',
 'reviewText_ExclQue_countchar',
 'summary_ExclQue_countchar',
 'reviewText_PunctChar_ratio',
 'summary_PunctChar_ratio',
 'unixReviewTime_delta_lastreview']

In [39]:
# print features that did not help improve
print(set(new_feature_list)-set(features_tokeep))

{'itemID_numReviews', 'summary_avgWordLength', 'reviewText_avgWordLength', 'reviewerID_numReviews', 'summary_reviewText_wordsRatio'}


In [40]:
sorted(feature_MAE, key= lambda x: x[1])

[('rating', 0.17534),
 ('rating_deviation', 0.17922),
 ('summary_posWordCount', 0.18122),
 ('reviewText_posWordRate', 0.18134),
 ('summary_posWordRate', 0.1817),
 ('summary_negWordCount', 0.18362),
 ('summary_negWordRate', 0.18408),
 ('reviewText_negWordRate', 0.18414),
 ('summary_count_firstCapital', 0.18484),
 ('unixReviewTime', 0.185),
 ('categoryID', 0.18502),
 ('reviewText_posWordCount', 0.18504),
 ('price', 0.18532),
 ('reviewText_negWordCount', 0.18544),
 ('reviewText_count_char', 0.18552),
 ('category_numtrans', 0.18574),
 ('reviewText_capitalwords', 0.18586),
 ('reviewText_count_firstCapital', 0.18588),
 ('unixReviewTime_delta_lastreview', 0.18594),
 ('reviewText_count_words', 0.186),
 ('reviewText_PunctChar_ratio', 0.1862),
 ('unixReviewTime_delta_firstreview', 0.18634),
 ('summary_count_char', 0.18658),
 ('summary_PunctChar_ratio', 0.18662),
 ('summary_count_punctu', 0.18678),
 ('reviewText_count_punctu', 0.1868),
 ('summary_reviewText_charRatio', 0.18692),
 ('categories_cou

## Apply PCA

In [28]:
# # normalize
# X_train_norm = X_train.apply(lambda x : (x-x.mean())/x.std())
# X_val_norm = X_val.apply(lambda x : (x-x.mean())/x.std())

In [29]:
# X_train_norm.shape

In [30]:
# %%time
# # apply pca
# pca = PCA(n_components=18)
# pca.fit(X_train_norm)
# X_train_transformed = pca.fit_transform(X_train_norm)
# X_val_transformed = pca.fit_transform(X_val_norm)

In [31]:
# X_train_transformed.shape

In [32]:
# # cumulative explained variance ratio
# import matplotlib.pyplot as plt

# print('Total Variance Explained Ratio: {}'.format(np.sum(pca.explained_variance_ratio_)))
# plt.plot(np.cumsum(pca.explained_variance_ratio_))
# plt.xlabel('number of components')
# plt.ylabel('cumulative explained variance');

## Try Different Models

### Random Forests

In [70]:
# randomized search CV
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 450, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [5, 11, 17, 23, 30, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [50, 150, 250, 350, 450]}


In [37]:
%%time
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 20.2min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 73.7min finished


CPU times: user 2min 51s, sys: 1.87 s, total: 2min 52s
Wall time: 1h 16min


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [92]:
%%time

from sklearn.model_selection import RandomizedSearchCV

# apply random forest for votes > 2
X_train = X_train_1[original_features + features_tokeep]
X_val = X_val_1[original_features + features_tokeep]

# Create the random grid
random_grid = {'n_estimators': [400,500,600],
               'max_depth': [8,10,12]
                }
 
rf = RandomForestRegressor()

regr = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 9 cv = 3, verbose=2, random_state=0, n_jobs = -1)

regr.fit(X_train, y_train)

y_pred = regr.predict(X_val)*np.array([i['outOf'] for i in X_val_raw['helpful']])
y_pred_train = regr.predict(X_train)*np.array([i['outOf'] for i in X_train_raw['helpful']])

# round to the nearest integer
y_pred = [int(round(i)) for i in y_pred]
y_pred_train = [int(round(i)) for i in y_pred_train]

print('md:{}, ne:{}'.format(md,ne))

print('Accuracy for train data: {}'.format(accuracy_score([i['nHelpful'] for i in y_train_raw],y_pred_train)))
print('Accuracy for val data: {}'.format(accuracy_score([i['nHelpful'] for i in y_val_raw],y_pred)))

print('MAE for train data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_train_raw], y_pred_train)))
print('MAE for val data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_val_raw], y_pred)))

/Users/gio/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 9 is smaller than n_iter=27. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 33.6min finished


md:12, ne:500
Accuracy for train data: 0.6317658025744263
Accuracy for val data: 0.870475
MAE for train data: 0.43832682123802535
MAE for val data: 0.16515
CPU times: user 4min 24s, sys: 1.58 s, total: 4min 26s
Wall time: 37min 55s


In [94]:
regr.best_params_

{'n_estimators': 400, 'max_depth': 12}

In [42]:
%%time

# apply random forest for votes > 2
X_train_temp = X_train_filtered[original_features + features_tokeep]
# X_val_temp = X_val_1[original_features]

for md in [10]:
    for ne in [100]:

        regr = RandomForestRegressor(max_depth=md,n_estimators=ne, random_state=0, n_jobs=-1, criterion='mse')
        # regr = RandomForestRegressor(n_estimators=450, min_samples_split=2, min_samples_leaf=2, random_state=0, 
        #                             max_features='auto', max_depth=11, bootstrap=True)
        regr.fit(X_train_temp, y_train_filtered)

        y_pred = regr.predict(X_val[original_features + features_tokeep])*np.array([i['outOf'] for i in X_val_raw['helpful']])
        y_pred_train = regr.predict(X_train[original_features + features_tokeep])*np.array([i['outOf'] for i in X_train_raw['helpful']])

        # round to the nearest integer
        y_pred = [int(round(i)) for i in y_pred]
        y_pred_train = [int(round(i)) for i in y_pred_train]

        print('md:{}, ne:{}'.format(md,ne))
        
        print('Accuracy for train data: {}'.format(accuracy_score([i['nHelpful'] for i in y_train_raw],y_pred_train)))
        print('Accuracy for val data: {}'.format(accuracy_score([i['nHelpful'] for i in y_val_raw],y_pred)))
        
        print('MAE for train data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_train_raw], y_pred_train)))
        print('MAE for val data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_val_raw], y_pred)))

md:10, ne:100
Accuracy for train data: 0.86622
Accuracy for val data: 0.86768
MAE for train data: 0.18704666666666667
MAE for val data: 0.16884
CPU times: user 1min 26s, sys: 1.13 s, total: 1min 27s
Wall time: 26.9 s


In [46]:
X_train.head()

,itemID,reviewerID,outOf_feature,rating,unixReviewTime,price,categoryID,categories_count,category_numtrans,summary_count_words,...,reviewText_count_firstCapital,summary_count_firstCapital,unixReviewTime_delta_firstreview,summary_reviewText_charRatio,itemID_numReviews,summary_capitalwords,summary_ExclQue_countchar,reviewText_PunctChar_ratio,summary_PunctChar_ratio,unixReviewTime_delta_lastreview
130248,15265,20193,13,4.0,1329004800,-999.00,0,2,551,3,...,24,1,139881600,0.022203,49,0,0,0.021349,0.038462,4924800
182454,10110,20969,1,2.0,1404691200,22.99,0,1,437,3,...,4,1,29894400,0.234783,5,0,0,0.043478,0.111111,1036800
176029,2813,6219,1,5.0,1376092800,12.99,0,1,327,1,...,3,0,26352000,0.076433,10,0,1,0.025478,0.083333,172800
13049,11768,14780,6,5.0,1383177600,59.99,0,1,1104,1,...,8,1,1123200,0.024922,5,0,0,0.037383,0.000000,86400
25017,5445,1962,1,1.0,1296604800,19.88,0,2,2276,2,...,7,1,5097600,0.057471,4,0,0,0.030651,0.066667,5097600


In [47]:
X_train_raw.outOf_feature

130248    13
182454     1
176029     1
13049      6
25017      1
          ..
140296     9
174074     2
16241      2
37597      4
181568     8
Name: outOf_feature, Length: 50419, dtype: int64

In [145]:
%%time

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# apply logistic regression to votes equal to 0 and 1

X_train = X_train_1[original_features + features_tokeep]
X_val = X_val_1[original_features + features_tokeep]

# parameters = { 'penalty': ['l1','l2'], 
#               'C':[0.1, 0.5, 1, 2, 3, 4, 5, 10]}
clf = LogisticRegression(penalty='l2', C=1, n_jobs=-1)
# clf = GridSearchCV(logreg, parameters, verbose=True, n_jobs=-1)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_val)*np.array([i['outOf'] for i in X_val_raw['helpful']])
y_pred_train = clf.predict(X_train)*np.array([i['outOf'] for i in X_train_raw['helpful']])

# round to the nearest integer
y_pred = [int(round(i)) for i in y_pred]
y_pred_train = [int(round(i)) for i in y_pred_train]

# print('md:{}, ne:{}'.format(md,ne))

print('Accuracy for train data: {}'.format(accuracy_score([i['nHelpful'] for i in y_train_raw],y_pred_train)))
print('Accuracy for val data: {}'.format(accuracy_score([i['nHelpful'] for i in y_val_raw],y_pred)))

print('MAE for train data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_train_raw], y_pred_train)))
print('MAE for val data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_val_raw], y_pred)))

Accuracy for train data: 0.8843714130369117
Accuracy for val data: 0.8773821388662378
MAE for train data: 0.11562858696308827
MAE for val data: 0.12261786113376219
CPU times: user 931 ms, sys: 92 ms, total: 1.02 s
Wall time: 4.76 s


In [140]:
X_train

,itemID,reviewerID,outOf_feature,rating,categories_count,reviewText_count_words,reviewerID_numReviews,reviewText_posWordCount,reviewText_count_char,reviewText_count_firstCapital,summary_avgWordLength
62732,16904,18201,0,5.0,3,17,7,0,219,5,4.0
182454,2023,28682,1,2.0,1,9,18,0,115,4,5.0
158745,1940,11231,0,4.0,1,25,9,0,255,9,4.0
176029,12868,14824,1,5.0,1,14,3,4,157,3,6.0
64520,6313,12493,0,5.0,3,22,5,6,249,8,6.0
...,...,...,...,...,...,...,...,...,...,...,...
9372,15188,37367,0,5.0,1,8,14,2,87,3,6.5
105595,13476,2891,0,4.0,1,9,5,3,102,5,4.0
93553,17844,30564,0,4.0,5,10,3,3,99,3,4.0
94735,16321,21837,0,3.0,2,8,5,4,111,4,2.0


In [146]:
X_train.shape

(132424, 11)

In [82]:
%%time
# USING PCA

regr = RandomForestRegressor(n_estimators=450, min_samples_split=2, min_samples_leaf=2, random_state=0, 
                            max_features='auto', max_depth=11, bootstrap=True)
regr.fit(X_train_transformed, y_train)

y_pred = regr.predict(X_val_transformed)*np.array([i['outOf'] for i in X_val_raw['helpful']])
y_pred_train = regr.predict(X_train_transformed)*np.array([i['outOf'] for i in X_train_raw['helpful']])

# round to the nearest integer
y_pred = [int(round(i)) for i in y_pred]
y_pred_train = [int(round(i)) for i in y_pred_train]

print('MAE for train data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_train_raw], y_pred_train)))
print('MAE for val data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_val_raw], y_pred)))

MAE for train data: 0.5872984390805054
MAE for val data: 0.189175
CPU times: user 2min 17s, sys: 475 ms, total: 2min 17s
Wall time: 2min 18s


In [140]:
%%time
# USING PCA

regr = RandomForestRegressor(max_depth=10, random_state=0)
regr.fit(X_train_transformed, y_train)

y_pred = regr.predict(X_val_transformed)*np.array([i['outOf'] for i in X_val_raw['helpful']])
y_pred_train = regr.predict(X_train_transformed)*np.array([i['outOf'] for i in X_train_raw['helpful']])

# round to the nearest integer
y_pred = [int(round(i)) for i in y_pred]
y_pred_train = [int(round(i)) for i in y_pred_train]

print('MAE for train data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_train_raw], y_pred_train)))
print('MAE for val data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_val_raw], y_pred)))

MAE for train data: 0.5644895773418751
MAE for val data: 0.19455
CPU times: user 1min 15s, sys: 328 ms, total: 1min 15s
Wall time: 1min 16s


### Decision Trees


In [100]:
%%time
for md in [1,2,3,4,5,6,7,8,9,10,100]:
    regr = DecisionTreeRegressor(max_depth=md, random_state=0)
    regr.fit(X_train, y_train)

    y_pred = regr.predict(X_val)*np.array([i['outOf'] for i in X_val_raw['helpful']])
    y_pred_train = regr.predict(X_train)*np.array([i['outOf'] for i in X_train_raw['helpful']])

    # round to the nearest integer
    y_pred = [int(round(i)) for i in y_pred]
    y_pred_train = [int(round(i)) for i in y_pred_train]

    print('max depth: {}'.format(md))
    print('MAE for train data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_train_raw], y_pred_train)))
    print('MAE for val data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_val_raw], y_pred)))

max depth: 1
MAE for train data: 0.7409111644419762
MAE for val data: 0.228675
max depth: 2
MAE for train data: 0.624407465439616
MAE for val data: 0.191625
max depth: 3
MAE for train data: 0.5931692417541007
MAE for val data: 0.179825
max depth: 4
MAE for train data: 0.5688728455542553
MAE for val data: 0.1715
max depth: 5
MAE for train data: 0.5607013229139809
MAE for val data: 0.1689
max depth: 6
MAE for train data: 0.5589162815605229
MAE for val data: 0.1689
max depth: 7
MAE for train data: 0.5465003272575815
MAE for val data: 0.167325
max depth: 8
MAE for train data: 0.5397766714928895
MAE for val data: 0.167175
max depth: 9
MAE for train data: 0.5195660366131816
MAE for val data: 0.16975
max depth: 10
MAE for train data: 0.5061187250837977
MAE for val data: 0.172975
max depth: 100
MAE for train data: 0.0
MAE for val data: 0.2337
CPU times: user 7.36 s, sys: 184 ms, total: 7.55 s
Wall time: 7.63 s


In [193]:
# %%time
# # USING PCA
# for md in [5, 10, 15]:
#     regr = DecisionTreeRegressor(max_depth=md, random_state=0)
#     regr.fit(X_train_transformed, y_train)

#     y_pred = regr.predict(X_val_transformed)*np.array([i['outOf'] for i in X_val_raw['helpful']])
#     y_pred_train = regr.predict(X_train_transformed)*np.array([i['outOf'] for i in X_train_raw['helpful']])

#     # round to the nearest integer
#     y_pred = [int(round(i)) for i in y_pred]
#     y_pred_train = [int(round(i)) for i in y_pred_train]

#     print('max depth: {}'.format(md))
#     print('MAE for train data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_train_raw], y_pred_train)))
#     print('MAE for val data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_val_raw], y_pred)))

### Multi-Layer Perceptron Regressor

In [38]:
for i in [100]:
    for j in [0.0001,0.01]:
        regr = MLPRegressor(random_state=1, max_iter=i, alpha=j)
        regr.fit(X_train, y_train)

        y_pred = regr.predict(X_val)*np.array([i['outOf'] for i in X_val_raw['helpful']])
        y_pred_train = regr.predict(X_train)*np.array([i['outOf'] for i in X_train_raw['helpful']])

        # round to the nearest integer
        y_pred = [int(round(i)) for i in y_pred]
        y_pred_train = [int(round(i)) for i in y_pred_train]

        print('max_iter: {}, alpha: {}'.format(i,j))
        print('MAE for train data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_train_raw], y_pred_train)))
        print('MAE for val data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_val_raw], y_pred)))

max_iter: 100, alpha: 0.0001
MAE for train data: 14318.49576875
MAE for val data: 14191.3678
max_iter: 100, alpha: 0.01
MAE for train data: 34262.94459375
MAE for val data: 33958.750725


### Least Squares

In [41]:
# Least squares regression
theta,residuals,rank,s = np.linalg.lstsq(X_train, y_train)

# Make predictions
y_pred = np.dot(X_val, theta)*np.array([i['outOf'] for i in X_val_raw['helpful']])
y_pred_train = np.dot(X_train, theta)*np.array([i['outOf'] for i in X_train_raw['helpful']])

# round to the nearest integer
y_pred = [int(round(i)) for i in y_pred]
y_pred_train = [int(round(i)) for i in y_pred_train]

print('MAE for train data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_train_raw], y_pred_train)))
print('MAE for val data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_val_raw], y_pred)))

/Users/gio/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


MAE for train data: 0.89464375
MAE for val data: 0.975725


### Linear Regression

In [47]:
regr = LinearRegression()
regr.fit(X_train, y_train)

y_pred = regr.predict(X_val)*np.array([i['outOf'] for i in X_val_raw['helpful']])
y_pred_train = regr.predict(X_train)*np.array([i['outOf'] for i in X_train_raw['helpful']])

# round to the nearest integer
y_pred = [int(round(i)) for i in y_pred]
y_pred_train = [int(round(i)) for i in y_pred_train]

print('MAE for train data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_train_raw], y_pred_train)))
print('MAE for val data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_val_raw], y_pred)))

MAE for train data: 0.8164625
MAE for val data: 0.878375


### Boosting


In [159]:
%%time
# Random Forests
X_train = X_train_1[original_features + features_tokeep]
X_val = X_val_1[original_features + features_tokeep]

regr = AdaBoostRegressor(RandomForestRegressor(max_depth=12,n_estimators=120, random_state=0), 
                         n_estimators = 120, random_state=10)
regr.fit(X_train, y_train)

y_pred = regr.predict(X_val)*np.array([i['outOf'] for i in X_val_raw['helpful']])
y_pred_train = regr.predict(X_train)*np.array([i['outOf'] for i in X_train_raw['helpful']])

# round to the nearest integer
y_pred = [int(round(i)) for i in y_pred]
y_pred_train = [int(round(i)) for i in y_pred_train]

print('MAE for train data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_train_raw], y_pred_train)))
print('MAE for val data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_val_raw], y_pred)))

MAE for train data: 0.47206410281838196
MAE for val data: 0.1754
CPU times: user 7min 58s, sys: 2.3 s, total: 8min
Wall time: 8min 4s


In [44]:
%%time
# Decision Trees
for i in [10,100]:

    regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=10, random_state=0), n_estimators = i, random_state=10)
    regr.fit(X_train, y_train)

    y_pred = regr.predict(X_val)*np.array([i['outOf'] for i in X_val_raw['helpful']])
    y_pred_train = regr.predict(X_train)*np.array([i['outOf'] for i in X_train_raw['helpful']])

    # round to the nearest integer
    y_pred = [int(round(i)) for i in y_pred]
    y_pred_train = [int(round(i)) for i in y_pred_train]

    print('MAE for train data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_train_raw], y_pred_train)))
    print('MAE for val data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_val_raw], y_pred)))

MAE for train data: 0.18184375
MAE for val data: 0.19145
MAE for train data: 0.19706875
MAE for val data: 0.207225
CPU times: user 2min 15s, sys: 3.13 s, total: 2min 18s
Wall time: 2min 19s


### Save test data predictions

In [49]:
%%time

from itertools import compress

# get indices
idx = np.array([i['outOf'] for i in df_features.helpful]) > 0

# Get features
X_train = Get_features(df_features)
X_test = Get_features(test_df)

# find filtered test data
X_train_filt = Get_features(df_features.loc[idx,:])
y_train_filt = Get_labels(list(compress(df_labels, idx))) 


CPU times: user 3min 58s, sys: 5.83 s, total: 4min 4s
Wall time: 4min 7s


In [46]:
# # remove reviews where votes are zero - training data only
# from itertools import compress

# idx = X_train.outOf_feature != 0

# X_train = X_train.loc[idx,:]
# y_train = list(compress(y_train, idx))

# df_features = df_features.loc[idx,:]

In [102]:
X_train

,categoryID,categories_count,category_numtrans,summary_count_words,itemID,reviewerID,rating,reviewText_count_words,itemID_helpfulRate,reviewerID_helpfulRate,...,summary_reviewText_charRatio,summary_reviewText_wordsRatio,itemID_numReviews,reviewText_capitalwords,summary_capitalwords,reviewText_ExclQue_countchar,summary_ExclQue_countchar,reviewText_PunctChar_ratio,summary_PunctChar_ratio,unixReviewTime_delta_lastreview
2,0,3,1672,2,1958,22815,3.0,15,0.850045,0.850045,...,0.118056,0.133333,7,0,0,0,0,0.048611,0.176471,5788800
4,0,3,1801,2,10744,19800,5.0,43,0.850045,0.850045,...,0.025943,0.046512,4,1,0,1,0,0.047170,0.000000,17625600
5,0,2,1374,2,7557,17278,4.0,44,0.850045,0.850045,...,0.051502,0.045455,50,2,0,0,0,0.021459,0.000000,950400
14,3,2,989,4,2451,2578,4.0,44,0.850045,0.850045,...,0.060453,0.090909,2,2,0,0,0,0.027708,0.000000,172800
20,0,4,2630,3,15427,10285,5.0,14,0.850045,0.850045,...,0.143939,0.214286,5,1,0,0,0,0.022727,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199979,0,1,1146,2,16056,901,5.0,12,0.850045,0.850045,...,0.158333,0.166667,4,1,0,0,0,0.041667,0.000000,0
199984,0,1,445,2,11341,13277,5.0,42,0.850045,0.850045,...,0.042654,0.047619,6,2,1,0,0,0.014218,0.055556,0
199992,1,3,2095,3,14605,22718,4.0,42,0.850045,0.850045,...,0.083333,0.071429,3,1,0,1,0,0.019380,0.000000,0
199996,1,1,1372,4,15611,17021,2.0,83,0.850045,0.850045,...,0.050132,0.048193,11,1,0,0,0,0.026385,0.026316,7430400


In [103]:
def save_predictions(pred):
    
    predictions = open("predictions_Helpful.txt", 'w')
    
    count = 0
    for l in open("pairs_Helpful.txt"):
        
        if l.startswith("userID"):
            #header``````````a
            predictions.write(l)
            continue
        
        u,i,outOf = l.strip().split('-')
        
        predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(pred[count]) + '\n')
        
        count+=1
    
    predictions.close()

In [111]:
%%time

from sklearn.model_selection import RandomizedSearchCV

# apply random forest for votes > 2
X_train = X_train[original_features + features_tokeep]
X_test = X_test[original_features + features_tokeep]

# Create the random grid
random_grid = {'n_estimators': [50,100,150,200,400,600],
               'max_depth': [8,10,12,14]
                }
 
rf = RandomForestRegressor()

regr = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20,cv = 5, verbose=2, random_state=0, n_jobs = -1)

regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)*np.array([i['outOf'] for i in test_df['helpful']])
y_pred_train = regr.predict(X_train)*np.array([i['outOf'] for i in df_features['helpful']])

y_pred = [int(round(i)) for i in y_pred]
y_pred_train = [int(round(i)) for i in y_pred_train]

# print(C)
print('Accuracy for train data: {}'.format(accuracy_score([i['nHelpful'] for i in df_features['helpful']], 
                                                          y_pred_train)))
print('MAE for train data: {}'.format(mean_absolute_error([i['nHelpful'] for i in df_features['helpful']], 
                                                          y_pred_train)))

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 38.4min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 108.4min finished


Accuracy for train data: 0.6700361812872921
MAE for train data: 0.3850926748762219
CPU times: user 9min 46s, sys: 3.04 s, total: 9min 49s
Wall time: 1h 57min 45s


In [112]:
regr.best_params_

{'n_estimators': 600, 'max_depth': 14}

In [50]:
%%time

X_train_temp = X_train_filt[original_features + features_tokeep]
# X_test = X_test[original_features + features_tokeep]

regr = RandomForestRegressor(max_depth=10, n_estimators=100, random_state=0)
regr.fit(X_train_temp, y_train_filt)

y_pred = regr.predict(X_test[original_features + features_tokeep])*np.array([i['outOf'] for i in test_df['helpful']])
y_pred_train = regr.predict(X_train[original_features + features_tokeep])*np.array([i['outOf'] for i in df_features['helpful']])

y_pred = [int(round(i)) for i in y_pred]
y_pred_train = [int(round(i)) for i in y_pred_train]

# print(C)
print('Accuracy for train data: {}'.format(accuracy_score([i['nHelpful'] for i in df_features['helpful']], 
                                                          y_pred_train)))
print('MAE for train data: {}'.format(mean_absolute_error([i['nHelpful'] for i in df_features['helpful']], 
                                                          y_pred_train)))


Accuracy for train data: 0.8683
MAE for train data: 0.176825
CPU times: user 1min 12s, sys: 847 ms, total: 1min 12s
Wall time: 1min 13s


In [90]:
%%time
regr = RandomForestRegressor(n_estimators=450, min_samples_split=2, min_samples_leaf=2, random_state=0, 
                            max_features='auto', max_depth=11, bootstrap=True)
regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)*np.array([i['outOf'] for i in test_df['helpful']])
y_pred_train = regr.predict(X_train)*np.array([i['outOf'] for i in df_features['helpful']])

y_pred = [int(round(i)) for i in y_pred]
y_pred_train = [int(round(i)) for i in y_pred_train]

# print(C)
print('MAE for train data: {}'.format(mean_absolute_error([i['nHelpful'] for i in df_features['helpful']], 
                                                          y_pred_train)))

MAE for train data: 0.4795924844483941
CPU times: user 3min 23s, sys: 1.34 s, total: 3min 25s
Wall time: 3min 31s


In [115]:
save_predictions(y_pred)